In [24]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from coffea import util
from coffea.processor import accumulate
import os, sys
import re
import pandas as pd
import hist
import matplotlib as mpl
import mplhep
from pathlib import Path

import utils

sys.path.append("../../analysisTools/")
import plotTools as ptools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
def add_signal_info_to_df(df):
    m1_list = []
    delta_list = []
    ctau_list = []
    
    for point in df.index.values:
        sig_dict = ptools.signalPoint(point)
        m1_list.append(sig_dict['m1'])
        delta_list.append(sig_dict['delta'])
        ctau_list.append(sig_dict['ctau'])
    
    df['m1'] = m1_list
    df['delta'] = delta_list
    df['ctau'] = ctau_list
    
    df = df.sort_values(by=['m1']) # sort by m1

    return df

In [124]:
def save_df_to_csv(df, outdir, outname, isSignal = False):
    Path(outdir).mkdir(parents=True, exist_ok=True)

    if isSignal:
        df = add_signal_info_to_df(df)
    
    df.to_csv(f'{outdir}/{outname}.csv')

    print(f'Saved: {outdir}/{outname}.csv')

# SR Cut-based: All Lxy (do not split SR by Lxy)

## Background

In [115]:
prefix = './coffea/skimmed/'
suffix = '_SR_AllLxy_skimmed'

In [116]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [117]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000002,0.000001,0.000001,0.000000,0.000000
DY,0.000011,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000007,0.000003,0.000002,0.000000,0.000000
Triboson,0.001933,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000034,0.000017,0.000011,0.000008,0.000002
W+jets,0.002310,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000027,0.000011,0.000009,0.000003,0.000001
Z+jets,0.006645,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000065,0.000019,0.000019,0.000005,0.000002
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [118]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_Cut-based_Background_Efficiency_AllLxyBin'

df_to_save = bkg_eff_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/Efficiency//SR_Cut-based_Background_Efficiency_AllLxyBin.csv


In [119]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
QCD,"38,039.83","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,0.00,0.00,0.00,0.00,0.00
Top,"5,667.49",559.22,349.76,314.03,312.99,312.95,294.00,88.48,32.85,27.62,2.18,1.05
DY,"14,495.72",671.44,623.25,623.25,623.25,623.25,455.65,143.51,143.51,143.51,143.51,0.00
Diboson,"4,424.20",307.91,227.24,213.36,211.27,210.33,189.35,49.47,20.79,15.55,2.81,1.09
Triboson,53.00,4.32,2.85,2.51,2.51,2.50,2.09,0.92,0.46,0.29,0.22,0.05
W+jets,"227,212.60","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39","2,620.46","1,064.21",874.29,249.49,88.23
Z+jets,"142,220.15","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76","1,384.45",409.60,400.98,113.21,46.46
Total,"432,113.00","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50","4,287.30","1,671.41","1,462.24",511.42,136.87


In [120]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_Cut-based_Background_EventCount_Weighted_AllLxyBin'

df_to_save = bkg_cts_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_Cut-based_Background_EventCount_Weighted_AllLxyBin.csv


In [121]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
QCD,12311,381,160,159,159,158,3,0,0,0,0,0
Top,49818,4688,3058,2348,2337,2336,2006,527,206,168,30,11
DY,205,13,11,11,11,11,9,1,1,1,1,0
Diboson,18067,1234,915,860,852,848,767,197,80,62,11,4
Triboson,2257,170,116,106,105,104,92,30,11,7,4,1
W+jets,697657,48425,37015,35472,35415,35377,29090,6940,3064,2457,724,256
Z+jets,1281459,67427,52627,50986,50934,50903,44480,10078,3096,3011,886,360
Total,2061774,122338,93902,89942,89813,89737,76447,17773,6458,5706,1656,632


In [122]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_Cut-based_Background_EventCount_Raw_AllLxyBin'

df_to_save = bkg_cts_raw_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Raw//SR_Cut-based_Background_EventCount_Raw_AllLxyBin.csv


In [123]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 136.87


## Signal

In [125]:
prefix = './coffea/skimmed/'
suffix = '_skimmed'

In [126]:
# Signal
sig_histo = util.load(f'{prefix}/signal_m1_5to50_SR_AllLxy_skimmed.coffea')[0]

In [127]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.5,1.0,10.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-10
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.5,1.0,100.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-100
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.5,1.0,1.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-1
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.2,0.5,100.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-100
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.2,0.5,1.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-1
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.2,0.5,10.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-10
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,55.0,10.0,100.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-100
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,55.0,10.0,10.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-10
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,55.0,10.0,1.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-1
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,52.5,5.0,100.0,50.0,55.0,0.1,sig_2018_Mchi-52p5_dMchi-5p0_ctau-100


In [128]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'Vertex chi2/df < 3',
 'dPhi(MET,vtx) < 1',
 'm(ee) < 20',
 'dR(ee) < 1.0',
 'minDxy > 0.01']

### Efficiency

In [129]:
pd.options.display.float_format = '{:,.6f}'.format

In [130]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [131]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [132]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,0.014484,0.001842,0.001519,0.001421,0.001418,0.001418,0.001404,0.000731,0.000714,0.000694,0.000653,0.000408
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,0.014782,0.002143,0.001894,0.001811,0.001811,0.001811,0.001794,0.000860,0.000814,0.000798,0.000710,0.000370
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,0.033826,0.006982,0.005961,0.005695,0.005695,0.005695,0.005651,0.003516,0.003294,0.003294,0.002804,0.001559
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,0.036318,0.006259,0.005421,0.005217,0.005217,0.005217,0.005197,0.002882,0.002601,0.002566,0.002171,0.001248
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,0.026904,0.005099,0.004306,0.004165,0.004161,0.004161,0.004135,0.002474,0.002306,0.002273,0.001869,0.001076
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,0.030555,0.005051,0.004338,0.004133,0.004127,0.004127,0.004103,0.002291,0.002131,0.002110,0.001845,0.001063
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,0.020878,0.003848,0.003369,0.003242,0.003235,0.003235,0.003195,0.001949,0.001844,0.001819,0.001615,0.000912
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,0.023214,0.003481,0.002980,0.002881,0.002881,0.002881,0.002855,0.001482,0.001394,0.001390,0.001214,0.000702
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,0.016920,0.002932,0.002531,0.002386,0.002386,0.002386,0.002357,0.001354,0.001306,0.001285,0.001175,0.000715
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,0.018942,0.002826,0.002381,0.002275,0.002273,0.002273,0.002253,0.001158,0.001101,0.001095,0.001000,0.000595


In [133]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_Cut-based_Signal_ctau-1_samples_Efficiency_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_Cut-based_Signal_ctau-1_samples_Efficiency_AllLxyBin.csv


In [134]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,0.010930,0.001187,0.000987,0.000935,0.000931,0.000931,0.000876,0.000373,0.000342,0.000318,0.000273,0.000228
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,0.010676,0.001022,0.000847,0.000792,0.000792,0.000792,0.000771,0.000318,0.000275,0.000271,0.000224,0.000173
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,0.031125,0.005252,0.004562,0.004329,0.004329,0.004329,0.004260,0.002492,0.002233,0.002207,0.001716,0.001414
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,0.032445,0.004564,0.003903,0.003752,0.003752,0.003752,0.003708,0.001840,0.001624,0.001607,0.001350,0.001136
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,0.024702,0.003809,0.003226,0.003090,0.003090,0.003090,0.003027,0.001676,0.001526,0.001512,0.001212,0.001000
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,0.025875,0.003520,0.003022,0.002902,0.002902,0.002902,0.002875,0.001426,0.001331,0.001323,0.001156,0.000973
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,0.019309,0.002550,0.002202,0.002122,0.002120,0.002120,0.002075,0.001145,0.001082,0.001072,0.000958,0.000826
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,0.020128,0.002757,0.002373,0.002303,0.002301,0.002301,0.002280,0.001063,0.000972,0.000960,0.000890,0.000758
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,0.015838,0.001936,0.001685,0.001584,0.001584,0.001584,0.001549,0.000792,0.000750,0.000738,0.000660,0.000580
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,0.016286,0.001989,0.001734,0.001669,0.001669,0.001669,0.001649,0.000730,0.000662,0.000657,0.000582,0.000472


In [135]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_Cut-based_Signal_ctau-10_samples_Efficiency_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_Cut-based_Signal_ctau-10_samples_Efficiency_AllLxyBin.csv


In [136]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.007414,0.000647,0.000562,0.000528,0.000528,0.000528,0.000501,0.000140,0.000099,0.000089,0.000058,0.000048
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,0.007306,0.000596,0.000504,0.000475,0.000475,0.000475,0.000457,0.000121,0.000075,0.000074,0.000044,0.000032
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,0.022439,0.002420,0.002063,0.001980,0.001980,0.001980,0.001893,0.000728,0.000568,0.000562,0.000388,0.000359
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,0.022522,0.002231,0.001905,0.001847,0.001847,0.001847,0.001804,0.000639,0.000505,0.000494,0.000406,0.000363
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,0.017576,0.001802,0.001476,0.001411,0.001411,0.001411,0.001364,0.000523,0.000434,0.000420,0.000321,0.000287
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,0.017664,0.001657,0.001419,0.001371,0.001371,0.001371,0.001331,0.000467,0.000376,0.000371,0.000295,0.000266
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,0.013585,0.001285,0.001082,0.001052,0.001052,0.001052,0.001011,0.000354,0.000300,0.000295,0.000239,0.000218
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,0.013536,0.001197,0.001041,0.000995,0.000995,0.000995,0.000966,0.000320,0.000241,0.000234,0.000175,0.000153
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,0.010335,0.000922,0.000793,0.000760,0.000760,0.000760,0.000722,0.000265,0.000210,0.000207,0.000156,0.000137
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,0.010649,0.000928,0.000818,0.000775,0.000775,0.000775,0.000749,0.000227,0.000170,0.000166,0.000113,0.000096


In [137]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_Cut-based_Signal_ctau-100_samples_Efficiency_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_Cut-based_Signal_ctau-100_samples_Efficiency_AllLxyBin.csv


### Weighted Event count

In [138]:
pd.options.display.float_format = '{:,.2f}'.format

In [139]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [140]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [141]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,"2,748.24",349.57,288.31,269.62,268.98,268.98,266.40,138.71,135.48,131.61,123.88,77.45
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"62,382.76","9,045.04","7,993.01","7,642.38","7,642.38","7,642.38","7,572.31","3,628.61","3,435.92","3,365.75","2,997.90","1,560.50"
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,205.19,42.35,36.16,34.55,34.55,34.55,34.28,21.33,19.98,19.98,17.01,9.45
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,"6,562.28","1,130.93",979.54,942.72,942.72,942.72,939.12,520.80,470.01,463.72,392.27,225.55
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,512.94,97.21,82.10,79.40,79.34,79.34,78.84,47.16,43.97,43.34,35.62,20.51
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,"16,193.18","2,676.90","2,298.73","2,190.27","2,187.08","2,187.08","2,174.33","1,214.08","1,129.50","1,118.34",977.97,563.12
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,"49,723.24","9,164.66","8,024.74","7,722.24","7,704.44","7,704.44","7,609.52","4,641.26","4,392.00","4,332.78","3,846.19","2,172.47"
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,"33,794.92","5,067.47","4,338.18","4,193.56","4,193.56","4,193.56","4,156.59","2,157.37","2,029.66","2,022.94","1,767.60","1,021.60"
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,995.45,172.47,148.91,140.36,140.36,140.36,138.67,79.65,76.85,75.58,69.13,42.07
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,"30,374.51","4,531.00","3,817.78","3,648.14","3,644.60","3,644.60","3,612.88","1,857.56","1,765.70","1,755.10","1,603.20",953.42


In [142]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_Cut-based_Signal_ctau-1_samples_EventCount_Weighted_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_Cut-based_Signal_ctau-1_samples_EventCount_Weighted_AllLxyBin.csv


In [143]:
# without m1 = 30 GeV
ct = 1.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,"2,748.24",349.57,288.31,269.62,268.98,268.98,266.40,138.71,135.48,131.61,123.88,77.45
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"62,382.76","9,045.04","7,993.01","7,642.38","7,642.38","7,642.38","7,572.31","3,628.61","3,435.92","3,365.75","2,997.90","1,560.50"
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,205.19,42.35,36.16,34.55,34.55,34.55,34.28,21.33,19.98,19.98,17.01,9.45
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,"6,562.28","1,130.93",979.54,942.72,942.72,942.72,939.12,520.80,470.01,463.72,392.27,225.55
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,512.94,97.21,82.10,79.40,79.34,79.34,78.84,47.16,43.97,43.34,35.62,20.51
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,"16,193.18","2,676.90","2,298.73","2,190.27","2,187.08","2,187.08","2,174.33","1,214.08","1,129.50","1,118.34",977.97,563.12
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,995.45,172.47,148.91,140.36,140.36,140.36,138.67,79.65,76.85,75.58,69.13,42.07
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,"30,374.51","4,531.00","3,817.78","3,648.14","3,644.60","3,644.60","3,612.88","1,857.56","1,765.70","1,755.10","1,603.20",953.42
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,"39,395.49","5,811.03","5,006.12","4,654.25","4,648.89","4,648.89","4,616.98","2,282.24","2,138.37","2,111.68","1,887.59","1,045.09"
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,"1,394.26",207.15,178.78,170.89,170.89,170.89,169.82,89.39,82.78,81.50,76.81,46.94


In [144]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,211.62,22.98,19.10,18.10,18.03,18.03,16.97,7.22,6.61,6.15,5.28,4.41
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,"6,090.24",583.27,483.38,451.73,451.73,451.73,439.55,181.40,157.05,154.63,127.85,98.62
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,18.96,3.20,2.78,2.64,2.64,2.64,2.60,1.52,1.36,1.34,1.05,0.86
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,625.95,88.06,75.30,72.38,72.38,72.38,71.54,35.51,31.33,31.00,26.04,21.91
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,47.30,7.30,6.18,5.92,5.92,5.92,5.80,3.21,2.92,2.90,2.32,1.91
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,"1,569.13",213.48,183.29,176.01,176.01,176.01,174.34,86.48,80.72,80.26,70.10,59.02
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,"15,555.45","2,054.32","1,773.84","1,709.70","1,707.69","1,707.69","1,671.59",922.02,871.90,863.89,771.62,665.42
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,"27,263.50","3,734.67","3,214.54","3,119.89","3,117.26","3,117.26","3,088.33","1,439.95","1,316.39","1,300.61","1,206.00","1,027.22"
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,94.00,11.49,10.00,9.40,9.40,9.40,9.20,4.70,4.45,4.38,3.92,3.44
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,"3,043.72",371.80,324.16,312.00,312.00,312.00,308.26,136.37,123.78,122.85,108.84,88.30


In [145]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_Cut-based_Signal_ctau-10_samples_EventCount_Weighted_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_Cut-based_Signal_ctau-10_samples_EventCount_Weighted_AllLxyBin.csv


In [146]:
# without m1 = 30 GeV
ct = 10.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,211.62,22.98,19.10,18.10,18.03,18.03,16.97,7.22,6.61,6.15,5.28,4.41
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,"6,090.24",583.27,483.38,451.73,451.73,451.73,439.55,181.40,157.05,154.63,127.85,98.62
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,18.96,3.20,2.78,2.64,2.64,2.64,2.60,1.52,1.36,1.34,1.05,0.86
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,625.95,88.06,75.30,72.38,72.38,72.38,71.54,35.51,31.33,31.00,26.04,21.91
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,47.30,7.30,6.18,5.92,5.92,5.92,5.80,3.21,2.92,2.90,2.32,1.91
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,"1,569.13",213.48,183.29,176.01,176.01,176.01,174.34,86.48,80.72,80.26,70.10,59.02
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,94.00,11.49,10.00,9.40,9.40,9.40,9.20,4.70,4.45,4.38,3.92,3.44
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,"3,043.72",371.80,324.16,312.00,312.00,312.00,308.26,136.37,123.78,122.85,108.84,88.30
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,118.48,13.27,10.81,10.19,10.19,10.19,9.84,4.46,4.04,3.99,3.53,2.76
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,"3,591.89",398.72,340.80,322.41,322.41,322.41,313.43,141.45,126.63,124.84,107.77,92.06


In [147]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,14.29,1.25,1.08,1.02,1.02,1.02,0.96,0.27,0.19,0.17,0.11,0.09
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,443.66,36.19,30.61,28.82,28.82,28.82,27.73,7.33,4.58,4.50,2.67,1.92
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,1.37,0.15,0.13,0.12,0.12,0.12,0.12,0.04,0.03,0.03,0.02,0.02
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,43.76,4.34,3.70,3.59,3.59,3.59,3.50,1.24,0.98,0.96,0.79,0.70
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,3.35,0.34,0.28,0.27,0.27,0.27,0.26,0.10,0.08,0.08,0.06,0.05
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,108.83,10.21,8.74,8.45,8.45,8.45,8.20,2.88,2.31,2.29,1.82,1.64
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,"1,302.77",123.25,103.80,100.85,100.85,100.85,96.92,33.96,28.80,28.30,22.89,20.92
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,"24,707.11","2,184.60","1,899.41","1,816.96","1,816.96","1,816.96","1,763.83",584.00,439.61,427.61,319.44,279.87
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6.14,0.55,0.47,0.45,0.45,0.45,0.43,0.16,0.12,0.12,0.09,0.08
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,200.85,17.50,15.42,14.61,14.61,14.61,14.12,4.29,3.21,3.14,2.13,1.81


In [148]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_Cut-based_Signal_ctau-100_samples_EventCount_Weighted_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_Cut-based_Signal_ctau-100_samples_EventCount_Weighted_AllLxyBin.csv


In [149]:
# without m1 = 30 GeV
ct = 100.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,14.29,1.25,1.08,1.02,1.02,1.02,0.96,0.27,0.19,0.17,0.11,0.09
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,443.66,36.19,30.61,28.82,28.82,28.82,27.73,7.33,4.58,4.50,2.67,1.92
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,1.37,0.15,0.13,0.12,0.12,0.12,0.12,0.04,0.03,0.03,0.02,0.02
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,43.76,4.34,3.70,3.59,3.59,3.59,3.50,1.24,0.98,0.96,0.79,0.70
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,3.35,0.34,0.28,0.27,0.27,0.27,0.26,0.10,0.08,0.08,0.06,0.05
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,108.83,10.21,8.74,8.45,8.45,8.45,8.20,2.88,2.31,2.29,1.82,1.64
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6.14,0.55,0.47,0.45,0.45,0.45,0.43,0.16,0.12,0.12,0.09,0.08
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,200.85,17.50,15.42,14.61,14.61,14.61,14.12,4.29,3.21,3.14,2.13,1.81
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,7.60,0.57,0.48,0.46,0.46,0.46,0.44,0.16,0.12,0.11,0.07,0.07
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,238.69,19.21,16.19,15.29,15.29,15.29,14.98,4.84,3.11,3.05,2.01,1.70


### Raw count

In [150]:
pd.options.display.float_format = '{:,.1f}'.format

In [151]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [152]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [153]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,4261,542,447,418,417,417,413,215,210,204,192,120
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,3558,516,456,436,436,436,432,207,196,192,171,89
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,761,157,134,128,128,128,127,79,74,74,63,35
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,14604,2517,2180,2098,2098,2098,2090,1159,1046,1032,873,502
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,8179,1550,1309,1266,1265,1265,1257,752,701,691,568,327
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,10151,1678,1441,1373,1371,1371,1363,761,708,701,613,353
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,8377,1544,1352,1301,1298,1298,1282,782,740,730,648,366
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,10056,1508,1291,1248,1248,1248,1237,642,604,602,526,304
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,7098,1230,1062,1001,1001,1001,989,568,548,539,493,300
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,8600,1283,1081,1033,1032,1032,1023,526,500,497,454,270


In [154]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_Cut-based_Signal_ctau-1_samples_EventCount_Raw_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_Cut-based_Signal_ctau-1_samples_EventCount_Raw_AllLxyBin.csv


In [155]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,3168,344,286,271,270,270,254,108,99,92,79,66
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5002,479,397,371,371,371,361,149,129,127,105,81
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,3609,609,529,502,502,502,494,289,259,256,199,164
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,13492,1898,1623,1560,1560,1560,1542,765,675,668,561,472
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,9065,1398,1184,1134,1134,1134,1111,615,560,555,445,367
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,10341,1407,1208,1160,1160,1160,1149,570,532,529,462,389
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,7762,1025,885,853,852,852,834,460,435,431,385,332
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,10374,1421,1223,1187,1186,1186,1175,548,501,495,459,391
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,8280,1012,881,828,828,828,810,414,392,386,345,303
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,6515,796,694,668,668,668,660,292,265,263,233,189


In [156]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_Cut-based_Signal_ctau-10_samples_EventCount_Raw_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_Cut-based_Signal_ctau-10_samples_EventCount_Raw_AllLxyBin.csv


In [157]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,2177,190,165,155,155,155,147,41,29,26,17,14
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,10655,869,735,692,692,692,666,176,110,108,64,46
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,6993,754,643,617,617,617,590,227,177,175,121,112
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,15529,1539,1314,1274,1274,1274,1244,441,348,341,280,250
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,10574,1084,888,849,849,849,821,315,261,253,193,173
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,16226,1522,1303,1259,1259,1259,1222,429,345,341,271,244
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,5296,501,422,410,410,410,394,138,117,115,93,85
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,14389,1272,1106,1058,1058,1058,1027,340,256,249,186,163
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6241,557,479,459,459,459,436,160,127,125,94,83
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,8195,714,629,596,596,596,576,175,131,128,87,74


In [158]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_Cut-based_Signal_ctau-100_samples_EventCount_Raw_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_Cut-based_Signal_ctau-100_samples_EventCount_Raw_AllLxyBin.csv


In [159]:
# for a given ctau and mass splitting
ct = 1.0
delta = 0.1

points = (si["ctau"] == ct) & (si["delta"] == delta)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,3558,516,456,436,436,436,432,207,196,192,171,89
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,14604,2517,2180,2098,2098,2098,2090,1159,1046,1032,873,502
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,10151,1678,1441,1373,1371,1371,1363,761,708,701,613,353
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,10056,1508,1291,1248,1248,1248,1237,642,604,602,526,304
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,8600,1283,1081,1033,1032,1032,1023,526,500,497,454,270
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,7390,1090,939,873,872,872,866,428,401,396,354,196
Total,54359,8592,7388,7061,7057,7057,7011,3723,3455,3420,2991,1714


## S/sqrt(B)

In [160]:
nBkg

136.87341354070207

In [161]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [162]:
pd.options.display.float_format = '{:,.4f}'.format

In [163]:
df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [164]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,4.4113,5.0000,0.2000,10.0000,136.8734
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.0918,5.0000,0.2000,100.0000,136.8734
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,77.4454,5.0000,0.2000,1.0000,136.8734
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,1.9156,5.0000,0.1000,100.0000,136.8734
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"1,560.4991",5.0000,0.1000,1.0000,136.8734
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,98.6241,5.0000,0.1000,10.0000,136.8734
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,1.6983,10.0000,0.1000,100.0000,136.8734
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,"1,045.0934",10.0000,0.1000,1.0000,136.8734
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,2.7580,10.0000,0.2000,10.0000,136.8734
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,0.0651,10.0000,0.2000,100.0000,136.8734


In [165]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0000,0.2000,10.0000,4.4113,136.8734,0.3771
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.0000,0.2000,100.0000,0.0918,136.8734,0.0078
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.0000,0.2000,1.0000,77.4454,136.8734,6.6197
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.0000,0.1000,100.0000,1.9156,136.8734,0.1637
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.0000,0.1000,1.0000,"1,560.4991",136.8734,133.3841
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.0000,0.1000,10.0000,98.6241,136.8734,8.4299
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,10.0000,0.1000,100.0000,1.6983,136.8734,0.1452
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,10.0000,0.1000,1.0000,"1,045.0934",136.8734,89.3296
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,10.0000,0.2000,10.0000,2.7580,136.8734,0.2357
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,10.0000,0.2000,100.0000,0.0651,136.8734,0.0056


In [166]:
outdir = './csv/SR_Yields/'
outname = 'SR_Cut-based_AllLxy_S_over_B'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/SR_Yields//SR_Cut-based_AllLxy_S_over_B.csv


### ctau = 1mm signal points

In [167]:
# S/sqrt(B) in ctau = 1mm, delta = 0.1
ct = 1.0
delta = 0.1

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.0000,0.1000,1.0000,"1,560.4991",136.8734,133.3841
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,10.0000,0.1000,1.0000,"1,045.0934",136.8734,89.3296
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,20.0000,0.1000,1.0000,953.4201,136.8734,81.4938
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,30.0000,0.1000,1.0000,"1,021.5963",136.8734,87.3212
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,40.0000,0.1000,1.0000,563.1172,136.8734,48.1326
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,50.0000,0.1000,1.0000,225.5517,136.8734,19.2791


In [168]:
# S/sqrt(B) in ctau = 1mm, delta = 0.2
ct = 1.0
delta = 0.2

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.0000,0.2000,1.0000,77.4454,136.8734,6.6197
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,10.0000,0.2000,1.0000,46.9415,136.8734,4.0123
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,20.0000,0.2000,1.0000,42.0688,136.8734,3.5958
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,30.0000,0.2000,1.0000,"2,172.4702",136.8734,185.6925
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,40.0000,0.2000,1.0000,20.5075,136.8734,1.7529
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,50.0000,0.2000,1.0000,9.4543,136.8734,0.8081


### ctau = 10mm signal points

In [169]:
# S/sqrt(B) in ctau = 10mm, delta = 0.1
ct = 10.0
delta = 0.1

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.0000,0.1000,10.0000,98.6241,136.8734,8.4299
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.0000,0.1000,10.0000,92.0583,136.8734,7.8687
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,20.0000,0.1000,10.0000,88.2994,136.8734,7.5474
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,30.0000,0.1000,10.0000,"1,027.2200",136.8734,87.8019
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,40.0000,0.1000,10.0000,59.0217,136.8734,5.0449
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,50.0000,0.1000,10.0000,21.9078,136.8734,1.8726


In [170]:
# S/sqrt(B) in ctau = 10mm, delta = 0.2
ct = 10.0
delta = 0.2

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0000,0.2000,10.0000,4.4113,136.8734,0.3771
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,10.0000,0.2000,10.0000,2.7580,136.8734,0.2357
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,20.0000,0.2000,10.0000,3.4396,136.8734,0.2940
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,30.0000,0.2000,10.0000,665.4163,136.8734,56.8766
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,40.0000,0.2000,10.0000,1.9148,136.8734,0.1637
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,50.0000,0.2000,10.0000,0.8614,136.8734,0.0736


### ctau = 100mm signal points

In [171]:
# S/sqrt(B) in ctau = 100mm, delta = 0.1
ct = 100.0
delta = 0.1

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.0000,0.1000,100.0000,1.9156,136.8734,0.1637
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,10.0000,0.1000,100.0000,1.6983,136.8734,0.1452
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,20.0000,0.1000,100.0000,1.8143,136.8734,0.1551
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,30.0000,0.1000,100.0000,279.8746,136.8734,23.9224
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,40.0000,0.1000,100.0000,1.6365,136.8734,0.1399
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,50.0000,0.1000,100.0000,0.7045,136.8734,0.0602


In [172]:
# S/sqrt(B) in ctau = 100mm, delta = 0.2
ct = 100.0
delta = 0.2

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.0000,0.2000,100.0000,0.0918,136.8734,0.0078
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,10.0000,0.2000,100.0000,0.0651,136.8734,0.0056
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,20.0000,0.2000,100.0000,0.0817,136.8734,0.0070
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,30.0000,0.2000,100.0000,20.9179,136.8734,1.7880
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,40.0000,0.2000,100.0000,0.0548,136.8734,0.0047
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,50.0000,0.2000,100.0000,0.0219,136.8734,0.0019
